## Лабораторная работа 7

In [210]:
import pandas as pd
from nltk import word_tokenize
from Levenshtein import distance as lev
import nltk
import pymorphy2
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shurik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [211]:
recipes = pd.DataFrame(pd.read_csv('07_nlp_data/ru_recipes_sample.csv'))
recipes['description'].replace(r'[^А-Яа-яЁё0-9\s]', '', regex=True, inplace=True)
recipes['description'] = [value.lower() for value in recipes['description']]
recipes['description']

0       этот коктейль готовлю из замороженной клубники...
1                                         быстро и вкусно
2                сытный овощной салатик пальчики оближете
3       картофельное пюре и куриные котлеты  вкусная к...
4       вишневая наливка имеет яркий вишневый вкус кот...
                              ...                        
3462    для тех кто любит чечевицу вам сюда очень вкус...
3463    баклажановые фантазии продолжаются предлагаю в...
3464    мое любимое блюдо лазанья но кушать только фар...
3465    прошлым летом варила варенье из одуванчиков по...
3466     и три корочки хлеба  сделал заказ буратино в ...
Name: description, Length: 3467, dtype: object

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [218]:
data = pd.DataFrame(word_tokenize(" ".join(recipes['description']))).drop_duplicates()
data1 = [data.sample(2).values.tolist() for i in range(5)]
for i in data1:
    print(f"d({i[0][0]}, {i[1][0]}) = {lev(i[0][0], i[1][0])}")



d(морковь, переложенные) = 10
d(заветные, автору) = 5
d(песенку, рождественским) = 9
d(тушили, совместного) = 11
d(блинам, усы) = 6


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [213]:
def is_plagiarism(s1: str, s2: str) -> bool:
    str_1 = word_tokenize(s1)
    str_2 = word_tokenize(s2)
    Max = max(len(s1), len(s2))
    total_lev = []
    for word1 in str_1:
        count_levenstein = []

        for word2 in str_2:
            count_levenstein.append(lev(word1, word2))

        if 0 in count_levenstein or 1 in count_levenstein:
           total_lev.append(1)

        else:
            total_lev.append(0)

    Sum = sum(total_lev)
    return True if (Sum / Max) > 0.8 else False

In [214]:
print('Необходимые нам индексы таблицы с ID = 135488 и 851934: ', end='\n')
var = [index for index in range(len(list(recipes['url']))) if (list(recipes['url'])[index].split('/')[-2] == '135488' or list(recipes['url'])[index].split('/')[-2] == '851934')]
print(var)
print(recipes.iloc[0])
print(recipes.iloc[1])
print(recipes.iloc[958]['description'], recipes.iloc[1473]['description'])
print(is_plagiarism(recipes.iloc[958]['description'], recipes.iloc[1473]['description']))

Необходимые нам индексы таблицы с ID = 135488 и 851934: 
[958, 1473]
url                https://www.povarenok.ru/recipes/show/164365/
name                          Густой молочно-клубничный коктейль
ingredients    {'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...
description    этот коктейль готовлю из замороженной клубники...
Name: 0, dtype: object
url                  https://www.povarenok.ru/recipes/show/1306/
name                                                    Рулетики
ingredients    {'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...
description                                      быстро и вкусно
Name: 1, dtype: object
прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим оре

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [220]:
morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer("russian")
stem_lem_df = pd.DataFrame(columns=['word', 'stemmed_word', 'normalized_word'])
stem_lem_df['word'] = [i[0] for i in data.values.tolist()]
stem_lem_df['stemmed_word'] = stem_lem_df['word'].apply(lambda x: stemmer.stem(x))
stem_lem_df['normalized_word'] = stem_lem_df['word'].apply(lambda x: morph.parse(x)[0].normalized.word)
stem_lem_df = stem_lem_df.set_index('word')
stem_lem_df

,stemmed_word,normalized_word
word,,
этот,этот,этот
коктейль,коктейл,коктейль
готовлю,готовл,готовить
из,из,из
замороженной,заморожен,заморозить
...,...,...
сбывается,сбыва,сбываться
подружим,подруж,подружить
одинаковых,одинаков,одинаковый


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [217]:
stops = stopwords.words('russian')
pattern = r'\b(?:{})\b'.format('|'.join(stops))
recipes['description_no_stopwords'] = recipes['description'].str.replace(pattern, '')


count_word = Counter(word_tokenize(' '.join(recipes['description'])))
count, count_stop = sum(count_word.values()), sum([count_word[x] for x in stops])
word_counts_after = Counter(word_tokenize(' '.join(recipes['description_no_stopwords'])))

word_count = count_word.most_common(10)
word_count_after = word_counts_after.most_common(10)

print(count, count_stop, count_stop / count)
print(word_count)
print(word_count_after)

C:\Users\Shurik\AppData\Local\Temp\ipykernel_25508\2266447829.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['description_no_stopwords'] = recipes['description'].str.replace(pattern, '')


102978 33223 0.322622307677368
[('и', 5054), ('в', 2584), ('с', 1934), ('на', 1655), ('очень', 1607), ('не', 1517), ('из', 1006), ('я', 979), ('рецепт', 869), ('а', 863)]
[('очень', 1607), ('рецепт', 869), ('это', 734), ('блюдо', 524), ('вкусный', 461), ('просто', 436), ('вкусно', 375), ('приготовить', 344), ('вкус', 324), ('салат', 313)]


### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [ ]:
def get_df_vect(sent_words):
    cv = TfidfVectorizer(lowercase=True)
    cv.fit(sent_words)
    return pd.DataFrame([{i: cv.transform(sent_words).toarray()[0][cv.vocabulary_[i]] for i in cv.vocabulary_}])

In [ ]:
pancakes = recipes[recipes['name'].str.contains('оладьи')]
sample = pancakes.sample(n=5)
for i in range(5):
    print(f"Рецепт №{i+1}")
    print(get_df_vect(nltk.sent_tokenize(sample['description'].values[i])), end='\n\n')

Рецепт №1
    нежные   сочные   оладьи    очень  вкусные
0  0.27735  0.27735  0.27735  0.83205  0.27735

Рецепт №2
        лет  двадцать     назад   покупая       сыр  наткнулась        на  \
0  0.242536  0.121268  0.121268  0.121268  0.121268    0.121268  0.363803   

   рекламный    листок     этому  ...       все  ингредиенты   указаны  \
0   0.121268  0.121268  0.121268  ...  0.121268     0.121268  0.121268   

        500     грамм     фарша  готовила       три      раза    больше  
0  0.121268  0.121268  0.121268  0.121268  0.121268  0.121268  0.121268  

[1 rows x 57 columns]

Рецепт №3
     нежные   вкусные  оладушки      мужу     очень  нравятся
0  0.408248  0.408248  0.408248  0.408248  0.408248  0.408248

Рецепт №4
      супер   быстрый   вариант  завтрака       это      даже   быстрее  \
0  0.316228  0.316228  0.316228  0.316228  0.316228  0.316228  0.316228   

        чем   сварить   овсянку  
0  0.316228  0.316228  0.316228  

Рецепт №5
    вкусные    сытные  оладушки   

7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [ ]:
sample_desc = [sample.iloc[x]['description'] for x in range(5)]
vector = CountVectorizer().fit_transform(sample_desc).toarray()
df_pancakes = pd.DataFrame(cosine_similarity(vector), columns=sample['name'].values, index=sample['name'].values)
for i in range(5):
    df_pancakes.iloc[i][i] = 0
df_pancakes


,Нежные куриные оладьи,Куриные оладьи с сыром,Тыквенные оладьи,Бананово-кукурузные оладьи,Картофельные оладьи с семгой
Нежные куриные оладьи,0.000000,0.100901,0.566139,0.000000,0.104828
Куриные оладьи с сыром,0.100901,0.000000,0.099015,0.038348,0.091670
Тыквенные оладьи,0.566139,0.099015,0.000000,0.000000,0.308607
Бананово-кукурузные оладьи,0.000000,0.038348,0.000000,0.000000,0.000000
Картофельные оладьи с семгой,0.104828,0.091670,0.308607,0.000000,0.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [ ]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    print(sim_df[sim_df == sim_df.max().max()].stack().values[0])
    return sim_df[sim_df == sim_df.max().max()].stack().index.tolist()[0]
find_closest(df_pancakes)

0.566138517072298


('Нежные куриные оладьи', 'Тыквенные оладьи')

In [ ]:
print(
    sample[sample['name']==find_closest(df_pancakes)[0]]['description'].values[0],
    sample[sample['name']==find_closest(df_pancakes)[1]]['description'].values[0],
    sep="\n"
    )

0.566138517072298
0.566138517072298
нежные сочные оладьи и очень очень очень вкусные
нежные вкусные оладушки  мужу очень нравятся
